<a href="https://colab.research.google.com/github/GianRomani/Neural_search_engine/blob/main/Romani_DM_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Not working on Colab, it should work in local. It is needed to use GPU with multiprocess (in Flow)
%env JINA_MP_START_METHOD='spawn'
import os
import torch.multiprocessing as mp

print(os.getenv("JINA_MP_START_METHOD"))
mp.set_start_method('spawn', force=True)

env: JINA_MP_START_METHOD='spawn'
'spawn'


In [2]:
try: 
  #this will raise an error if you are not using Colab
  from google.colab import drive
  drive.mount('/content/drive')
except:
  print("You are not using Colab")

Mounted at /content/drive


In [3]:
import pandas as pd

In [4]:
questions = pd.read_csv("/content/drive/MyDrive/DM/Questions.csv", encoding="ISO-8859-1")
answers = pd.read_csv("/content/drive/MyDrive/DM/Answers.csv", encoding="ISO-8859-1")

index_outfile = "/content/drive/MyDrive/DM/index_1k.csv"

# EDA

It can be interesting and helpful to analyze:

*   the distribution of the scores among the questions,
*   number of answers for question (and scores of those answers);
*   creation dates for the posts;
*   if a post was closed (and why);
*   popular users (who answered a lot of questions)





<h2>Questions</h2>

*   Id: id of the question;
*   OwnerUserId: id of the account who post the question;
*   CreationDate: when the question was posted;
*   ClosedDate: when the question was closed;
*   Score: upvotes - downvotes;
*   Title: title of the question;
*   Body: the question

In [ ]:
questions.head(10)

In [ ]:
questions.describe()

In [ ]:
#What is the distribution of the scores among the questions?
score_distribution = questions["Score"].value_counts(bins=20)
score_distribution

In [ ]:
#negative scores
negative_scores_questions = questions[questions["Score"] < 0]
negative_scores_questions

In [ ]:
#Posts with a score greater than 4000
best_posts = questions[questions["Score"] >4000]
best_posts

In [ ]:
#Count NaN values -> most of them in ClosedDate
questions.isna().sum()

<h2>Answers</h2>


*   Id: id of the answer;
*   OwnerUserId: id of the account who post the answer;
*   CreationDate: when the answer was posted;
*   ParentId: Id of the question;
*   Score: upvotes - downvotes;
*   Body: the answer



In [ ]:
answers.head(10)

In [ ]:
answers.describe()

In [ ]:
#Distribution for the scores of the answers
score_distribution = answers["Score"].value_counts(bins=20)
score_distribution

In [ ]:
#negative scores
negative_scores_answers = answers[answers["Score"] < 0]
negative_scores_answers

In [ ]:
#Number of answers for question
answers_for_post = answers.groupby(["ParentId"]).count()["Id"]
answers_for_post

In [ ]:
#max number of answers for a post
answers_for_post.max()

In [ ]:
#How many questions have a certain number of answers?
print("#answers   #questions")
answers_for_post.value_counts()

In [ ]:
#How many NaN values?
answers.isna().sum()

In [ ]:
#How many questions were answered by the users?
users_and_number_of_answers = answers["OwnerUserId"].value_counts()
print("UserId     #answers")
users_and_number_of_answers

In [ ]:
#Distribution of the number of answers posted by different users
print("#answers    #users")
users_and_number_of_answers.value_counts()

<h2>Tags</h2>


*   Id: id of the question
*   Tag: just the tag :)



In [ ]:
tags.head(10)

In [ ]:
#How many rows?
tags.shape[0]

In [ ]:
#Check if there are some null values
tags.isna().sum()

In [ ]:
#Number of tags for post
number_of_tags_for_post = tags["Id"].value_counts()
print("Post     #tags")
number_of_tags_for_post

In [ ]:
#Distribution of number of tags
number_of_tags_for_post.hist()

In [ ]:
#Number of tags for post
tags_used = tags["Tag"].value_counts()
tags_used

In [ ]:
counter = 0
for i in tags_used:
  if i > 10:
    counter+=1
counter

In [ ]:
#How many tags appear a certain number of times
print("#occurrences #tags")
occ_tags = tags_used.value_counts()
occ_tags

In [ ]:
#distribution of how many tags appear a certain number of times
tags_used_groups = tags_used.value_counts(bins=20)
tags_used_groups

# Preprocessing

In [5]:
import re
from bs4 import BeautifulSoup as soup
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
def cleanpunc(sentence): 
    no_punct = sentence.translate(str.maketrans('', '', string.punctuation))
    return  no_punct

In [7]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"\n", " ", phrase)
    return phrase

In [8]:
stop_words = set(stopwords.words('english'))
print ('list of stop words:', stop_words)

def clean_stop_words(total_text):
    if type(total_text) is not int:
        string = ""
        for word in total_text.split():
            if not word in stop_words:
                string += word + " "
        return string  

list of stop words: {"wouldn't", 'o', "mustn't", 'ourselves', 'by', 'further', 'just', 'above', 'mightn', 'yourselves', 'over', 'through', 'we', 'don', 'should', 'during', 'a', "should've", 'it', 'more', "didn't", "doesn't", "it's", 'too', 'wouldn', 'as', 'any', 'he', 'themselves', 'the', 'his', 'didn', 'hers', 'again', "haven't", "isn't", 'have', 'under', 'are', 'myself', 'be', "needn't", "hadn't", 'will', 'being', 'me', 'having', 'am', "you're", 'she', 'him', 'against', 'why', "you've", 'such', 'y', 'about', 'own', 'your', 'm', 'nor', 'other', 'doesn', 'below', 'because', 'then', 'can', 'after', 'at', "mightn't", 'off', 'this', 'now', "shan't", 'both', 'hadn', 'were', "wasn't", 'but', 'than', 'ours', 'those', 'to', 'their', 'my', 'into', "couldn't", 'there', 'yourself', 'needn', 'yours', 'll', 'had', 'between', 'was', 'from', 'what', 'when', 'itself', 're', 'has', 'shan', 'out', 'himself', 'once', 'very', 's', 'how', 'theirs', 'for', 'been', 'doing', 'aren', 'mustn', 'haven', 'so', '

In [9]:
def preprocessing_operations(text):
  splitted_text_code = re.split('<pre><code>|</code></pre>', text)
  res = []
  for idx, text in enumerate(splitted_text_code):
    if idx%2 == 0:
      no_html = soup(text, "lxml").text
      no_punctuation = cleanpunc(no_html)
      no_contractions = decontracted(no_punctuation)
      no_stopwords = clean_stop_words(no_contractions)
      res.append(no_stopwords)
    else:
      res.append(text)
  return res

def preprocess_data(data):
  cleaned_text = []
  if type(data)==type("string"): #preprocess a single text
    cleaned_text = preprocessing_operations(data)
  else: #preprocess sevaral texts in input
    body = data["Body"]
    for q in body:
      cleaned_text.append(preprocessing_operations(q))
  return cleaned_text

# Jina

In [10]:
!pip install jina

     |████████████████████████████████| 864 kB 5.5 MB/s 
     |████████████████████████████████| 3.6 MB 43.8 MB/s 
     |████████████████████████████████| 10.9 MB 39.5 MB/s 
     |████████████████████████████████| 596 kB 50.3 MB/s 
     |████████████████████████████████| 52 kB 569 kB/s 
     |████████████████████████████████| 217 kB 53.3 MB/s 
     |████████████████████████████████| 146 kB 51.8 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
     |████████████████████████████████| 3.8 MB 44.2 MB/s 
     |████████████████████████████████| 1.1 MB 40.1 MB/s 
     |████████████████████████████████| 111 kB 48.6 MB/s 
     |████████████████████████████████| 1.8 MB 40.3 MB/s 
     |████████████████████████████████| 1.8 MB 44.5 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 58 kB 6.2 MB/s 
     |████████████████████████████████| 402 kB 51.0 MB/s 
     |████████████████████████████████| 144 kB 51.4 MB/s 
     |████████████████

In [11]:
! pip install "transformers" "pytorch-lightning" 

     |████████████████████████████████| 3.5 MB 5.3 MB/s 
     |████████████████████████████████| 527 kB 34.8 MB/s 
     |████████████████████████████████| 895 kB 44.8 MB/s 
     |████████████████████████████████| 6.8 MB 28.4 MB/s 
     |████████████████████████████████| 67 kB 2.8 MB/s 
     |████████████████████████████████| 397 kB 37.7 MB/s 
     |████████████████████████████████| 133 kB 39.2 MB/s 
     |████████████████████████████████| 829 kB 10.6 MB/s 
     |████████████████████████████████| 952 kB 20.0 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=228cb51bf360b0116d54f07ffaaa669950f7d9f548eb6525cb366d3b092b32bc
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: f

In [12]:
!apt install libomp-dev
!pip install faiss
!pip install --upgrade faiss-gpu

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11-0 cuda-nsight-compute-11-1
  cuda-nsight-systems-10-1 cuda-nsight-systems-

In [13]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn as nn
from __future__ import division
from math import ceil
import numpy as np
from jina import Flow, Executor, requests, Document, DocumentArray, DocumentArrayMemmap
import faiss
import time

import warnings
warnings.filterwarnings('ignore')

In [14]:
device = 0
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


The authors of CodeBert set the MAX_LENGTH for the sentences to 512(+2) so, if the input vector is too big, I can not compute the embeddings -> two possible solutions (https://github.com/microsoft/CodeBERT/issues/16):

*   use an RNN over k representations to obtain a final embedding;
*   take the average of the representations.

To use the cosine distance we need vectors that have same length ->there are two ways to obtain a representation of same length for the texts:

*   use the [CLS] token;
*   use the average of the vectors




In [15]:
class MyTransformer(Executor):
    """Transformer executor class """

    def __init__(
            self,
            pretrained_model_name_or_path: str = "microsoft/codebert-base",
            *args,
            device: str = 'cpu',
            **kwargs,
    ):
        super().__init__(*args, **kwargs)
        self.tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path)
        self.model = AutoModel.from_pretrained(pretrained_model_name_or_path)
        self.model.to(device)
        print("Device used: {}".format(self.model.device))

    def compute_embedding(self,tokens_ids):
      if(len(tokens_ids) < 512):  
        context_embeddings = self.model(torch.tensor(tokens_ids)[None,:].to(device))[0]
      else:
        k = ceil(len(tokens_ids) / 512.0)
        chunks = np.array_split(tokens_ids, k)
        chunks_tensor = torch.tensor(chunks[0])[None,:].to(device)
        context_embeddings_list = self.model(chunks_tensor)[0].to(device)
        for i in range(1, len(chunks)):
          chunks_tensor = torch.tensor(chunks[i])[None,:].to(device)
          context_embeddings_list = torch.cat((context_embeddings_list, self.model(chunks_tensor)[0]), 1)
        dim = context_embeddings_list.shape[-1]
        context_embeddings = context_embeddings_list.to(device)
      #compute the average of the vectors
      avg_embeddings = torch.mean(context_embeddings,1)[0]
      return avg_embeddings

    @requests
    def encode(self, docs: 'DocumentArray', *args, **kwargs):
        start = time.time()
        tokens = [self.tokenizer.cls_token]
        texts = docs.get_attributes("text")
        for idx, q in enumerate(texts): 
          cleaned_q = preprocess_data(q)
          tokens = [self.tokenizer.cls_token]
          for l in cleaned_q:
            code_tokens = self.tokenizer.tokenize(l)
            tokens += code_tokens
            tokens += [self.tokenizer.sep_token]
          tokens_ids = self.tokenizer.convert_tokens_to_ids(tokens)
          docs[idx].embedding = self.compute_embedding(tokens_ids)
        print("Time needed to encode {} documents: {}".format(len(docs), time.time()-start))

In [16]:
class MyIndexer(Executor): #Indexer with Faiss library

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.index_output = None
        self.index_file = None
        self.index_output = kwargs["index_output"]

    @requests(on='/index')
    def index(self, docs: DocumentArray, mode: str = None, **kwargs): 
      dim = docs.embeddings.shape[1]
      nlist = 50  # how many cells
      m = 8  # number of centroid IDs in final compressed vectors
      bits = 8 # number of bits in each centroid

      embeddings = docs.embeddings.cpu().detach().numpy()
      quantizer = faiss.IndexFlatL2(dim)
      self.my_index = faiss.IndexIVFPQ(quantizer, dim, nlist, m, bits) 

      start = time.time()
      self.my_index.train(embeddings)
      self.my_index.add(embeddings)
      print("Time needed to train and add {} texts to the index: {} s".format(self.my_index.ntotal, time.time()-start))

      if(mode == "w"):
        #write index on disk
        faiss.write_index(self.my_index, self.index_output)

    @requests(on='/search')
    def search(self, docs: DocumentArray, k:int = 4, **kwargs):
        s, d = self.my_index.search(docs.embeddings.numpy(), k)
        start = time.time()
        print("Time needed to search {} query: {} s".format(len(docs), time.time()-start))
        print("Indexes of the most similar documents: {}\nwith distance: {}".format(d,s))
        return d[0]
          
    def print_index(self):
      for d in self.index:
        print("Document id: {}\nembedding: {} of shape: {}".format(d.id, d.embedding, d.embedding.shape))

In [17]:
def getKey(item):
  return item[2]

def get_answers(docIds: list, questions, answers):
  for i, idx in enumerate(docIds):
    if idx == -1: #This result is not valid
      continue
    print("{}th result:".format(i+1))
    #document
    doc = questions.loc[idx]
    doc_id = doc.Id
    #question's title
    title = doc["Title"]
    #question's body
    body_q = doc["Body"]
    #answers
    answers_q = answers.loc[answers.ParentId == doc_id]
    answers.drop(answers[answers.Score <= 0].index, inplace=True)
    if (answers_q.empty):
      #skip this question on SO -> maybe add as feature the fact that other documents will be retrieved instead
      continue
    answers_texts = []
    for a in answers_q.itertuples():
      answers_texts.append((a.Id, a.Body, a.Score))
    answers_texts_sorted = sorted(answers_texts, key=getKey, reverse=True)
    print("Doc Id: {}\nTitle: {}\nText:\n{}".format(doc_id, title, body_q))
    for j, a in enumerate(answers_texts_sorted):
      print("----------------------------")
      print("{}th answer's Id: {}, with score: {} and text:\n{}".format(j+1, a[0], a[2], a[1]))
    print("########################################")

In [22]:
def test_with_gpu(num_docs: int = 100, query:str = None, **kwargs):
  start = time.time()
  #index
  i = MyIndexer(index_output = kwargs["index_output"], index_file = kwargs["index_file"]) #pass operation = 'w' to write and save a new index on disk
  if(num_docs == 0): #load the whole dataset
    docArray = DocumentArray([Document(id=questions.iloc[i].Id, text=questions.iloc[i].Body) for i in range(len(questions))])
    print("Time needed to read the whole dataset: {} s".format(time.time()-start))
  else:
    docArray = DocumentArray([Document(id=questions.iloc[i].Id, text=questions.iloc[i].Body) for i in range(num_docs)])
    print("Time needed to read {} texts: {} s".format(num_docs, time.time()-start))
    #embeddings
    e = MyTransformer(device=device)
    e.encode(docArray)
    #create new index
    i.index(docArray, mode="w") 
  #query
  if(query is None):
    q = DocumentArray([Document(text=questions.iloc[1].Body)])
    e.encode(q)
  else:
    q = DocumentArray([Document(text = query)])
    e.encode(q)
  #search
  docs = i.search(q, k=5)
  get_answers(docs, questions, answers)

In [19]:
flow = (
        Flow()
            .add(name='MyTransformer', uses=MyTransformer, uses_with={'device': "cuda:0"}, gpus='all')
            .add(name='MyIndexer', uses=MyIndexer)
    )

In [20]:
flow.plot()

           Flow@82[I]:flow visualization: https://mermaid.ink/svg/ICAgICAgICAgICAgJSV7aW5pdDp7ICAidGhlbWUiOiAiYmFzZSIsICAidGhlbWVWYXJpYWJsZXMiOiB7ICAgICAgInByaW1hcnlDb2xvciI6ICIjZmZmIiwgICAgICAicHJpbWFyeUJvcmRlckNvbG9yIjogIiNmZmYiLCAgICAgICJtYWluQmtnIjogIiMzMkM4Q0QiLCAgICAgICJjbHVzdGVyQmtnIjogIiNFRUVERTc4QyIsICAgICAgInNlY29uZGFyeUJvcmRlckNvbG9yIjogIm5vbmUiLCAgICAgICJ0ZXJ0aWFyeUJvcmRlckNvbG9yIjogIm5vbmUiLCAgICAgICJsaW5lQ29sb3IiOiAiI2E2ZDhkYSIgICAgICB9fX0lJSAgICAgICAgICAgIApmbG93Y2hhcnQgTFI7CnN1YmdyYXBoIE15VHJhbnNmb3JtZXI7Ck15VHJhbnNmb3JtZXIvcGVhLTBbTXlUcmFuc2Zvcm1lcl06OjpQRUE7CmVuZDsKc3ViZ3JhcGggTXlJbmRleGVyOwpNeUluZGV4ZXIvcGVhLTBbTXlJbmRleGVyXTo6OlBFQTsKZW5kOwpnYXRld2F5c3RhcnRbZ2F0ZXdheV06OjpHQVRFV0FZIC0tPiBNeVRyYW5zZm9ybWVyOjo6UE9EOwpNeVRyYW5zZm9ybWVyOjo6UE9EIC0tPiBNeUluZGV4ZXI6OjpQT0Q7Ck15SW5kZXhlcjo6OlBPRCAtLT4gZ2F0ZXdheWVuZFtnYXRld2F5XTo6OkdBVEVXQVk7CmNsYXNzRGVmIElOU1BFQ1Qgc3Ryb2tlOiNGMjlDOUYKY2xhc3NEZWYgSk9JTl9JTlNQRUNUIHN0cm9rZTojRjI5QzlGCmNsYXNzRGVmIEdBVEVXQVkgZmlsbDpub25lLGNvbG

In [26]:
query_encoder = MyTransformer(device=device)
num_docs = 1000 #number of documents to read, 0 to load the whole dataset

if(device.type!="cpu"): #if you are using GPU
  print("\nInsert your query ('n' to not pass a query): ")
  q = input()
  if(q=='n'): #use a question from the dataset as query
    test_with_gpu(num_docs, index_output = index_outfile, index_file = index_outfile)
  else:
    test_with_gpu(num_docs, q, index_output = index_outfile, index_file = index_outfile)
else: #if you are not using GPU
  with flow:
      docArray = DocumentArray([Document(id=questions.iloc[i].Id, text=questions.iloc[i].Body) for i in range(num_docs)])
      flow.index(docArray, index_output = index_outfile, index_file = index_outfile)
      while(True):
        print("\nInsert your query (q to quit): ")
        q = input()
        if (q == "q"):
          print("\n\nGoodbye!")
          break
        query_encoder.encode(DocumentArray([q]))
        d = flow.search(
            inputs=DocumentArray(q)
            )
        get_answers(d, questions, answers)

Device used: cuda:0

Insert your query ('n' to not pass a query): 
n
Time needed to read 1000 texts: 0.5496423244476318 s
Device used: cuda:0


Token indices sequence length is longer than the specified maximum sequence length for this model (2312 > 512). Running this sequence through the model will result in indexing errors


Time needed to encode 1000 documents: 29.743781805038452
Time needed to train and add 1000 texts to the index: 0.39182615280151367 s
Time needed to encode 1 documents: 0.016463756561279297
Time needed to search 1 query: 5.4836273193359375e-06 s
Indexes of the most similar documents: [[  1 509 588 407 535]]
with distance: [[4.066163  7.5142684 7.80306   7.922653  8.129744 ]]
1th result:
Doc Id: 90
Title: Good branching and merging tutorials for TortoiseSVN?
Text:
<p>Are there any really good tutorials explaining <a href="http://svnbook.red-bean.com/en/1.8/svn.branchmerge.html" rel="nofollow">branching and merging</a> with Apache Subversion? </p>

<p>All the better if it's specific to TortoiseSVN client.</p>

----------------------------
1th answer's Id: 1466832, with score: 19 and text:
<p>My easy click-by-click instructions (<strong>specific to TortoiseSVN</strong>) are in Stack&nbsp;Overflow question <em><a href="http://stackoverflow.com/questions/1461922">What is the simplest way to 